In [1]:
import os
import time
import chess
import chess.pgn
import numpy as np

In [3]:
start = time.time()

pgn = open('data/fics_202011_notime_50k.pgn')

def doNothing():
    return

while chess.pgn.read_game(pgn):
    doNothing()

end = time.time()
print(f'Time elapsed: {end - start}')

Time elapsed: 115.31920433044434


In [11]:
def encode_result(game):
    '''
    Returns encoding of game result as integer:
        0: white wins (1-0)
        1: black wins (0-1)
        2: draw (1/2-1/2)
    '''
    result = 0
    result_string = game.headers['Result']
    if result_string[0] == '0':
        result = 1
    elif '1/2' in result_string:
        result = 2
    return result

def encode_ending(movetext):
    '''
    Win / Lose:
        0: checkmated
        1: resigned
        2: timeout
    Draw:
        3: repetition
        4: agreement
        5: insufficient material
        6: stalemate
    '''
    ending = 0
    if 'resign' in movetext:
        ending = 1
    elif 'forfeits on time' in movetext:
        ending = 2
    elif 'drawn by repetition' in movetext:
        ending = 3
    elif 'drawn by mutual agreement' in movetext:
        ending = 4
    elif 'Neither player has mating material' in movetext:
        ending = 5
    elif 'drawn by stalemate' in movetext:
        ending = 6
    return ending
    

def game_features(game):
    '''
    Extracts features from game & game metadata
    '''
    result = encode_result(game)
    num_moves = game.end().ply()
    movetext = str(game.mainline())
    num_checks = movetext.count('+')
    num_kingside_castle = movetext.count('O-O')
    num_queenside_castle = movetext.count('O-O-O')
    num_pawn_promotion = movetext.count('=')
    ending = encode_ending(movetext)
    return [result, num_moves, num_checks, num_kingside_castle, num_queenside_castle, num_pawn_promotion, ending]

In [12]:
pgn = open('data/fics_202011_notime_50k.pgn')
game = chess.pgn.read_game(pgn)
game.headers

Headers(Event='FICS rated blitz game', Site='FICS freechess.org', Date='2020.11.30', Round='?', White='stevharr', Black='nimzoii', Result='1-0', FICSGamesDBGameNo='474470039', WhiteElo='1677', BlackElo='1337', WhiteRD='0.0', BlackRD='0.0', TimeControl='300+0', Time='23:49:00', WhiteClock='0:05:00.000', BlackClock='0:05:00.000', ECO='B32', PlyCount='117')

In [13]:
game_features(game)

[0, 117, 11, 2, 0, 1, 2]

In [15]:
# [pawn, knight, bishop, rook, queen], see https://en.wikipedia.org/wiki/Chess_piece_relative_value
piece_values = [1, 3, 3, 5, 9]

# Given chess.Board and chess.Color
# Returns sum of piece values for that color
def get_piece_value(board, color):
    piece_value_sum = 0
    for i in range(0, 5):
        piece_value_sum += piece_values[i] * len(board.pieces(i+1, color))
    return piece_value_sum


board = game.board()
get_piece_value(board, chess.BLACK)

39

In [17]:
# Given chess.pgn.Game and chess.Color
# Return -1 if draw, 1 if color won, 0 if color lost.
def get_game_result(game, color):
    if '1/2' in game.headers['Result']:
        return -1
    elif color == chess.WHITE:
        return game.headers['Result'][0]
    else:
        return game.headers['Result'][2]


white_elo = game.headers['WhiteElo']
black_elo = game.headers['BlackElo']
    
print(f'White Elo: {white_elo}, Black Elo: {black_elo}')
print(f'Result for white: {get_game_result(game, chess.WHITE)}')

White Elo: 1677, Black Elo: 1337
Result for white: 1


In [18]:
def board_to_vec(board):
    '''
        Given a chess.Board return a vector of length 64
        representing the piece / lack of piece at a given square.
    '''
    vec = np.zeros((64), dtype=int)
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece is not None:
            if piece.color == chess.WHITE:
                vec[square] = piece.piece_type
            else:
                vec[square] = -1 * piece.piece_type
    return vec

def game_to_vec(game, moves_limit):
    '''
    Given a chess.Game, return a concatenation of board states
    represented as vectors, as generated by board_to_vec()
    '''
    board = game.board()
    game_as_vec = np.zeros((64 * moves_limit))
    i = 0
    for move in game.mainline_moves():
        if i >= moves_limit:
            break
        board.push(move)
        game_as_vec[(64*i):(64*(i+1))] = board_to_vec(board)
    return game_as_vec

In [19]:
game_to_vec(game, 10)

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0., -1., -1.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0., -4.,  1.,  0.,  0.,  0.,  0.,  0., -6.,
        0.,  6.,  0.,  0.,  0.,  0.,  0.,  0.,  5.,  0.,  0.,  4.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0

In [20]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

def game_to_movetext(game, move_limit=-1):
    '''
    Returns a list of the moves of chess.Game 'game' as strings in
    Standard Algebraic Notation (https://en.wikipedia.org/wiki/Algebraic_notation_(chess))
    '''
    game_string = str(game.mainline())
    move_strings = game_string.split('. ')[1:move_limit]
    move_strings = list(map(lambda s: s.rsplit(' ', 1)[0], move_strings))
    flattened_move_strings = [move for sublist in move_strings for move in sublist]
    return flattened_move_strings

len(game_to_movetext(game))

422

In [21]:
game_string = str(game.mainline())
# Extract 'move' strings
move_strings = game_string.split('. ')[1:-1]
move_strings = list(map(lambda s: s.rsplit(' ', 1)[0], move_strings))
flattened_move_strings = [move for sublist in move_strings for move in sublist]
flattened_move_strings

['e',
 '4',
 ' ',
 'c',
 '5',
 'N',
 'f',
 '3',
 ' ',
 'N',
 'c',
 '6',
 'd',
 '4',
 ' ',
 'c',
 'x',
 'd',
 '4',
 'N',
 'x',
 'd',
 '4',
 ' ',
 'd',
 '6',
 'B',
 'e',
 '2',
 ' ',
 'g',
 '6',
 'B',
 'e',
 '3',
 ' ',
 'B',
 'g',
 '7',
 'O',
 '-',
 'O',
 ' ',
 'e',
 '6',
 'N',
 'c',
 '3',
 ' ',
 'N',
 'g',
 'e',
 '7',
 'f',
 '4',
 ' ',
 'O',
 '-',
 'O',
 'g',
 '4',
 ' ',
 'a',
 '6',
 'f',
 '5',
 ' ',
 'e',
 'x',
 'f',
 '5',
 'g',
 'x',
 'f',
 '5',
 ' ',
 'f',
 '6',
 'B',
 'c',
 '4',
 '+',
 ' ',
 'K',
 'h',
 '8',
 'N',
 'e',
 '6',
 ' ',
 'B',
 'x',
 'e',
 '6',
 'B',
 'x',
 'e',
 '6',
 ' ',
 'b',
 '5',
 'B',
 'f',
 '4',
 ' ',
 'N',
 'e',
 '5',
 'Q',
 'd',
 '4',
 ' ',
 'N',
 '7',
 'c',
 '6',
 'Q',
 'd',
 '2',
 ' ',
 'Q',
 'b',
 '6',
 '+',
 'B',
 'e',
 '3',
 ' ',
 'Q',
 'c',
 '7',
 'R',
 'a',
 'd',
 '1',
 ' ',
 'R',
 'a',
 'd',
 '8',
 'N',
 'd',
 '5',
 ' ',
 'Q',
 'b',
 '7',
 'b',
 '3',
 ' ',
 'N',
 'g',
 '4',
 'B',
 'b',
 '6',
 ' ',
 'R',
 'a',
 '8',
 'h',
 '3',
 ' ',
 'N',
 'g',
 'e',
 '5'